In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
pip install -q -U tensorflow-hub tensorflow-text tensorflow-addons


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 83.8 MB/s eta 0:00:00


In [ ]:
import os
import collections
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


## Projection head

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Load video embeddings
#def load_video_embeddings(video_num):
 #   return np.load(f"/content/drive/MyDrive/CS482-Milestone4/combined_embeddings_folder/video{video_num}_combined_embedding.npy") #video1_combined_embedding

def load_video_embeddings(video_num):
    # Load video embeddings
    #video_embeddings = np.load(f'/content/drive/MyDrive/CS482-Milestone4/combined_embeddings_folder/video{video_num}_combined_embedding.npy', allow_pickle=True)
    video_embeddings = np.load(f'/content/drive/MyDrive/CS482-Milestone4/TextPembed/video{video_num}.npy', allow_pickle=True)


    return video_embeddings


# Load text embeddings
def load_text_embeddings(video_num):
    #return np.load(f'/content/drive/MyDrive/CS482-Milestone4/TextPembed/video{video_num}.npy', allow_pickle=True)
    #text_embeddings = np.load(f'/content/drive/MyDrive/CS482-Milestone4/combined_embeddings_folder/video{video_num}_combined_embedding.npy', allow_pickle=True)
    text_embeddings = np.load(f'/content/drive/MyDrive/CS482-Milestone4/TextPembed/video{video_num}.npy', allow_pickle=True)

    return text_embeddings

# Reshape the input data to match the expected shape for Dense layers
def reshape_input_data(data):
    # Assuming data is one-dimensional, reshape it to (batch_size, features)
    reshaped_data = data.reshape(data.shape[0], -1)  # Replace -1 with the appropriate number of features

    return reshaped_data


In [ ]:
# Projection head function
def projection_head(video_embeddings, text_embeddings, projection_dim):
    # Implement your projection head logic using Dense layers or other desired methods
    # Transform video and text embeddings into the joint embedding space with the given dimensionality
    video_input = keras.layers.Input(shape=video_embeddings.shape[1:])
    text_input = keras.layers.Input(shape=text_embeddings.shape[1:])

    # Projection layers for video embeddings
    video_projection = keras.layers.Dense(projection_dim, activation='relu')(video_input)

    # Projection layers for text embeddings
    text_projection = keras.layers.Dense(projection_dim, activation='relu')(text_input)

    # Return the transformed video and text embeddings
    return video_projection, text_projection

In [ ]:
def projection_head(video_embeddings, text_embeddings, projection_dim):
    # Implement projection head logic using Dense layers or desired methods
    # Transform video and text embeddings into the joint embedding space with the given dimensionality

    # Assuming you're using Dense layers for projection
    video_input = keras.layers.Input(shape=video_embeddings.shape[1:])
    text_input = keras.layers.Input(shape=text_embeddings.shape[1:])

    # Projection layers for video embeddings
    projected_video = keras.layers.Dense(projection_dim, activation='relu')(video_input)

    # Projection layers for text embeddings
    projected_text = keras.layers.Dense(projection_dim, activation='relu')(text_input)

    # Reshape projected_video and projected_text to 2D
    projected_video = keras.layers.Flatten()(projected_video)
    projected_text = keras.layers.Flatten()(projected_text)

    return projected_video, projected_text


## Contrastive loss

In [ ]:
# Contrastive loss function
def contrastive_loss(y_true, y_pred, margin=1):
    # Calculate contrastive loss
    square_pred = tf.square(y_pred)
    margin_square = tf.square(tf.maximum(margin - y_pred, 0))
    return tf.reduce_mean((1 - y_true) * square_pred + y_true * margin_square)

# Prepare data for contrastive learning
def prepare_contrastive_data(video_embeddings, text_embeddings):
    # Implement logic to create pairs of video-text embeddings and their labels (similar or dissimilar)
    # For simplicity, create pairs randomly labeled as similar or dissimilar
    num_samples = min(video_embeddings.shape[0], text_embeddings.shape[0])
    labels = np.random.choice([0, 1], size=(num_samples,))
    return video_embeddings[:num_samples], text_embeddings[:num_samples], labels


## Train the model

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import EarlyStopping, LearningRateScheduler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA



#video_embeddings_folder = "/content/drive/MyDrive/CS482-Milestone4/combined_embeddings_folder"
video_embeddings_folder = "/content/drive/MyDrive/CS482-Milestone4/TextPembed"

# Load video embeddings
video_embeddings = []
for video_num in range(1, 51):
    #video_embedding = np.load(f'{video_embeddings_folder}/video{video_num}_combined_embedding.npy', allow_pickle=True)
    video_embedding = np.load(f'{video_embeddings_folder}/video{video_num}.npy', allow_pickle=True)

    video_embeddings.append(video_embedding)
text_embeddings_folder = "/content/drive/MyDrive/CS482-Milestone4/TextPembed"
#text_embeddings_folder = "/content/drive/MyDrive/CS482-Milestone4/combined_embeddings_folder"


# Load text embeddings
text_embeddings = []
for video_num in range(1, 51):
    text_embedding = np.load(f'{text_embeddings_folder}/video{video_num}.npy', allow_pickle=True)
    #text_embedding = np.load(f'{video_embeddings_folder}/video{video_num}_combined_embedding.npy', allow_pickle=True)

    text_embeddings.append(text_embedding)

# Combine all video embeddings
video_embeddings = np.concatenate(video_embeddings, axis=0)  # Use axis=0 to concatenate along the first axis

# Combine all text embeddings
text_embeddings = np.concatenate(text_embeddings, axis=0)  # Use axis=0 to concatenate along the first axis
print("Video Embeddings Shape:", video_embeddings.shape)
print("Text Embeddings Shape:", text_embeddings.shape)

# Split data into training and validation sets
X_train_video, X_val_video, X_train_text, X_val_text = train_test_split(video_embeddings, text_embeddings, test_size=0.2)

num_features = 256

# Reshape X_train_text and X_val_text
X_train_text = X_train_text.reshape((-1, num_features))
X_val_text = np.repeat(X_val_text, len(X_val_video) // len(X_val_text), axis=0)  # Repeat to match X_val_video size

# Reshape X_val_video and X_train_video
X_val_video = X_val_video.reshape((-1, num_features))
X_train_video = X_train_video.reshape((-1, num_features))

# Create dummy labels for training and validation
y_train_dummy = np.random.rand(X_train_video.shape[0])
y_val_dummy = np.random.rand(X_val_video.shape[0])


#Create the projection head model
projection_input_video = keras.layers.Input(shape=X_train_video[0].shape)
projection_input_text = keras.layers.Input(shape=X_train_text[0].shape)

projected_video = keras.layers.Dense(128, activation='relu')(projection_input_video)
projected_text = keras.layers.Dense(128, activation='relu')(projection_input_text)

projection_model = keras.Model(inputs=[projection_input_video, projection_input_text], outputs=[projected_video, projected_text])

# Create the contrastive learning model
contrastive_input_video = keras.layers.Input(shape=(128,))
contrastive_input_text = keras.layers.Input(shape=(128,))

contrastive_output = keras.layers.Concatenate()([contrastive_input_video, contrastive_input_text])
contrastive_output = keras.layers.Dense(1, activation='linear')(contrastive_output)

contrastive_model = keras.Model(inputs=[contrastive_input_video, contrastive_input_text], outputs=contrastive_output)

# Compile the contrastive learning model
contrastive_model.compile(optimizer='adam', loss=contrastive_loss)

# Create early stopping and learning rate scheduler callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
learning_rate_scheduler = LearningRateScheduler(lambda epoch: 0.001 * 0.5 ** (epoch // 10))

history = contrastive_model.fit(
    [X_train_video, X_train_text],
    y_train_dummy,
    epochs=50,
    validation_data=([X_val_video, X_val_text], y_val_dummy),
    callbacks=[early_stopping, learning_rate_scheduler]
)

plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()



Video Embeddings Shape: (25486, 300)
Text Embeddings Shape: (25486, 300)


ValueError: ignored